# Курсова работа по "Дълбоко самообучение с Тензорфлоу"

на Александър Игнатов, Ф№0MI3400082, 05.07.2022г.

# Тема: „Приложение на метода Deep Q-Learning (DQN) за играта Space Invaders за ретро игрови конзоли Atari 2600.“

## Избран обект от Gym

Обект на обучението е играта Space Invaders за Atari 2600.

![game](https://www.gymlibrary.ml/_images/space_invaders.gif)

Играчът контролира наземна ракета, която придвижва наляво или надясно, и от която може да стреля нагоре. Точки печели, когато унищожава противникови ракети и губи играта, когато те достигнат земята или го унищожат с изстрел. Целта е максимално натрупване на точки в рамките на три живота.


Gym ни предоставя модел на играта с три различни възможни начина на наблюдение на състоянието и в три различни версии (v0, v4, v5). Използвана от проекта версия е v4, като са проведени експерименти с два различни типа пространство на състоянието (наблюдение на пикселите на екрана и наблюдение на RAM паметта на играта) с цел сравнение на двата подхода.
V4 се характеризира с това, че по подразбиране всяко действие се задържа за 2, 3 или 4 кадъра от играта на случаен принцип. Това прави стохастичен иначе детерминистичния модел на играта.

In [3]:
import gym

При варианта с наблюдение на пикселите, състоянието на играта е тензор с размерности 210x160x3 (височина х ширина х цвят):

In [4]:
env = gym.make("SpaceInvaders-v4")
env.observation_space.shape

(210, 160, 3)

При варианта с наблюдение на RAM паметта, състоянието е вектор с дължина 128, понеже играта има само 128 байта в RAM паметта си:

In [6]:
envram = gym.make("SpaceInvaders-v4", obs_type="ram")
envram.observation_space.shape

(128,)

Възможните действия в играта на всеки кадър са 6 на брой:

In [7]:
env.unwrapped.get_action_meanings()

['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

При избор на случайно действие на всяка стъпка се постига резултат от около 150 точки в играта (долният експеримент изчислява средноаритметичния reward при 100 епизода при случайно избиране на действие на всяка стъпка):

In [8]:
import random
import numpy as np

EPISODES = 100
scores = []

for episode in range(1, EPISODES + 1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        action = random.choice(range(env.action_space.n))
        n_state, reward, done, info = env.step(action)
        score += reward
    
    scores.append(score)

avg = np.mean(scores)
print(f"Average reward: {avg}")
env.close()


Average reward: 155.75


## Избран метод за обучение

Избраният метод за обучение е Deep Q-Learning (DQN) с Experience Replay и $\epsilon$-greedy стратегия (https://arxiv.org/pdf/1312.5602.pdf).

Псевдокод на алгоритъма:
1. Инициализация на replay memory $D$ с капацитет $N$
2. Инициализация на Q-функцията със случайни тегла
3. За $episode = 1...M$, направи:
    * Инициализирай редица $ s_1 = \{x_1\} $
    * За $ t = 1... T $, направи:
        * С вероятност $\epsilon$ избери случайно действие $a_t$, в противен случай избери $ a_t = argmax_a Q^*(s_t, a; \theta) $
        * Изпълни действието в емулатора и наблюдавай награда $r_t$ и състояние $x_{t+1}$
        * Запази прехода $ (s_t, a_t, r_t, s_{t+1}) $ в $ D $
        * Вземи произволно малко количество от преходи $ (s_j , a_j , r_j , s_{j+1}) $ от $ D $
        * Присвои $$ y_j =
            \begin{cases}
            r_j & \text{за терминален } s_{j+1} \\
            r_j + \gamma \max_{a'} Q^*(s_{j+1}, a'; \theta) & \text{за нетерминален } s_{j+1}
            \end{cases} $$
        * Направи градиентно спускане по $ (y_j − Q(s_j, a_j; \theta))^2 $:
$$ \nabla_{\theta_i}L_i(\theta_i) = E_{s,a \sim p(·); s' \sim \epsilon} [(r + \gamma\max_a'Q(s', a'; \theta_{i-1}) - Q(s, a, \theta_i))\nabla_{\theta_i}Q(s, a; \theta_i)] $$
$$ L_i(\theta_i) = E_{s,a \sim p(·)}[(y_i - Q(s, a; \theta_i))^2] $$
$$ Q^*(s, a) = E_{s' \sim \epsilon}[r + \gamma\max_a'Q^*(s', a') \mid s, a] $$

## Реализация и ескперименти

In [ ]:
from tensorflow.keras import Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D, Resizing, Rescaling, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow.image import rgb_to_grayscale
from tensorflow.keras.layers import Layer
from tensorflow.keras.utils import register_keras_serializable
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

### 0. Функция за създаване на агента

Хиперпараметрите, които са използвани, са:
* Капацитет на experience replay паметта: 1 000 000
* Намаляне на $\epsilon$: от 1.0 до 0.1 в хода на 10 000 стъпки
* Брой стъпки преди започване на обучението: 1 000
* $\gamma = 0.99$
* Размер на batch: 32

In [10]:
def build_agent(model, actions, window_size):
    policy = LinearAnnealedPolicy(
        EpsGreedyQPolicy(), 
        attr='eps', 
        value_max=1.0, 
        value_min=0.1, 
        value_test=0.2, 
        nb_steps=10000
    )
    memory = SequentialMemory(
        limit=1000000, 
        window_length=window_size
    )
    dqn = DQNAgent(
        model=model, 
        memory=memory, 
        policy=policy,
        enable_dueling_network=True, 
        dueling_type='avg', 
        nb_actions=actions, 
        nb_steps_warmup=1000
    )
    return dqn

### 1. Наблюдение на пикселното изображение на играта

Стандартно за тази цел е използването на конволюционни слоеве в невронната мрежа с няколко филтри. Бе експериментирано с различни конфигурации на мрежата, като при всички тях времето за обучение бе изключително бавно. Поради това бе взето решение изображението с размер 210х160 да се намаля наполовина по двете измерения и конвертира в черно-бяло такова. Това намаля размерността на входящия тензор до 105х80х1, което значително сваля бройката на параметрите за обучение, но не даде забележим резултат за времето за обучение.

In [8]:
@register_keras_serializable("atari")
class GrayscaleLayer(Layer):
  def call(self, input):
    return rgb_to_grayscale(input)


In [19]:
def build_model(window_size, height, width, channels, actions):
    model = Sequential()
    model.add(Input(shape=(window_size, height, width, channels)))
    model.add(Reshape((window_size * height, width, channels), name="reshape_stack"))
    model.add(GrayscaleLayer(name="grayscale"))
    model.add(Resizing((window_size * height) // 2, width // 2, name="resize_half"))
    model.add(Rescaling(1./255, name="normalize")) # normalize to [0, 1]
    model.add(Reshape((window_size, height // 2, width // 2, 1), name="reshape_unstack"))
    model.add(Convolution2D(32, (8,8), strides=(4,4), activation='relu', name="conv1"))
    model.add(Convolution2D(64, (4,4), strides=(2,2), activation='relu', name="conv2"))
    model.add(Convolution2D(64, (3,3), activation='relu', name="conv3"))
    model.add(Flatten(name="flatten"))
    model.add(Dense(512, activation='relu', name="fully_connected_1"))
    # model.add(Dense(256, activation='relu'))
    model.add(Dense(actions, activation='linear', name="output"))
    return model

In [20]:
WINDOW_SIZE = 4
height, width, channels = env.observation_space.shape
actions = env.action_space.n

Използвайки `WINDOW_SIZE = 4` задаваме едновременната обработка на последните 4 състояния на играта на всяка стъпка. Това се прави с цел определяне на посоката на разитие.

In [12]:
model = build_model(WINDOW_SIZE, height, width, channels, actions)
model.summary()


Model: "seqmodel"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape (Reshape)           (None, 840, 160, 3)       0         
                                                                 
 grayscale (GrayscaleLayer)  (None, 840, 160, 1)       0         
                                                                 
 resizing (Resizing)         (None, 420, 80, 1)        0         
                                                                 
 rescaling (Rescaling)       (None, 420, 80, 1)        0         
                                                                 
 reshape_1 (Reshape)         (None, 4, 105, 80, 1)     0         
                                                                 
 conv2d (Conv2D)             (None, 4, 25, 19, 32)     2080      
                                                                 
 conv2d_1 (Conv2D)           (None, 4, 11, 8, 64)      328

Както се вижда, получената мрежа с три конволюционни слоя и един изцяло-свързан слой с 512 неврона съдържа малко над 7 милиона параметъра (тегла) за обучение, които дори и с видеокартите, предоставени от Google Collab са твърде много за да може обучението да се впише в предоставениете ми няколко часа прозорец за активност на платформата.

In [29]:
dqn = build_agent(model, actions, WINDOW_SIZE)
dqn.compile(Adam(learning_rate=0.00025))

In [24]:
dqn.fit(env, nb_steps=10000, visualize=False, verbose=0)

Training for 10000 steps ...
done, took 14326.813 seconds


In [25]:
scores = dqn.test(env, nb_episodes=20, visualize=False)
avg = np.mean(scores.history["episode_reward"])
print(f"Average reward: {avg}")

Testing for 20 episodes ...
Episode 1: reward: 40.000, steps: 694
Episode 2: reward: 230.000, steps: 826
Episode 3: reward: 25.000, steps: 570
Episode 4: reward: 60.000, steps: 792
Episode 5: reward: 35.000, steps: 557
Episode 6: reward: 20.000, steps: 661
Episode 7: reward: 225.000, steps: 967
Episode 8: reward: 115.000, steps: 844
Episode 9: reward: 80.000, steps: 677
Episode 10: reward: 85.000, steps: 545
Episode 11: reward: 75.000, steps: 672
Episode 12: reward: 90.000, steps: 1360
Episode 13: reward: 50.000, steps: 375
Episode 14: reward: 105.000, steps: 1188
Episode 15: reward: 80.000, steps: 683
Episode 16: reward: 15.000, steps: 497
Episode 17: reward: 340.000, steps: 988
Episode 18: reward: 10.000, steps: 734
Episode 19: reward: 65.000, steps: 435
Episode 20: reward: 20.000, steps: 407


Average reward: 88.25

Виждаме, че 10 000 стъпки са отнели 14326.813 секунди (почти 4 часа) и дали среден резултат от 88.25 точки на епизод, който е двойно по-лош от този при играта със случайни действия.

### 2. Наблюдение на RAM паметта на играта

Предимството на използването на паметта като пространство от състояния на тази ретро игра, вместо пикселното изображение, което човек вижда, е че тя е само 128 байта, което е едно доста по-лесно смилаемо число от гледна точка на машинното самообучение.

In [11]:
envram.observation_space.shape

(128,)

Това позволява за много по-проста структура на невронна мрежа, която е по-бърза за обучение и по-бърза за изпълнение. Един или два скрити слоя от неврони са достатъчни за тази цел.

In [48]:
def build_ram_model(ram_size, actions, window_size):
    model = Sequential(name="ram_model")
    model.add(Input(shape=(window_size, ram_size)))
    model.add(Flatten(name="flatten"))
    model.add(Dense(512, activation="relu", name="fc1"))
    model.add(Dense(128, activation="relu", name="fc2"))
    model.add(Dense(actions, activation="linear", name="output"))
    return model


In [50]:
WINDOW_SIZE = 4
ram_model = build_ram_model(envram.observation_space.shape[0], envram.action_space.n, WINDOW_SIZE)
ram_model.summary()

Model: "ram_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 512)               0         
                                                                 
 fc1 (Dense)                 (None, 512)               262656     
                                                                 
 fc2 (Dense)                 (None, 128)               65664     
                                                                 
 output (Dense)              (None, 6)                 774       
                                                                 
Total params: 329,094
Trainable params: 329,094
Non-trainable params: 0
_________________________________________________________________


Виждаме, че при използване на два слоя с по 512 и 128 неврона съответно, параметрите за обучение са сведени от милиони само до малко под 400 хиляди, което е значителна оптимизация.

In [53]:
ram_dqn = build_agent(ram_model, envram.action_space.n, WINDOW_SIZE)
ram_dqn.compile(Adam(learning_rate=0.0002))
ram_dqn.fit(envram, nb_steps=100_000, visualize=False, verbose=0)

Training for 100000 steps ...
done, took 913.856 seconds


In [55]:
scores = ram_dqn.test(envram, nb_episodes=20, visualize=False)
avg = np.mean(scores.history["episode_reward"])
print(f"Average reward: {avg}")

Testing for 20 episodes ...
Episode 1: reward: 225.000, steps: 898
Episode 2: reward: 485.000, steps: 1027
Episode 3: reward: 380.000, steps: 961
Episode 4: reward: 80.000, steps: 393
Episode 5: reward: 210.000, steps: 670
Episode 6: reward: 135.000, steps: 525
Episode 7: reward: 230.000, steps: 939
Episode 8: reward: 220.000, steps: 639
Episode 9: reward: 355.000, steps: 916
Episode 10: reward: 400.000, steps: 1018
Episode 11: reward: 125.000, steps: 637
Episode 12: reward: 155.000, steps: 582
Episode 13: reward: 110.000, steps: 468
Episode 14: reward: 415.000, steps: 1119
Episode 15: reward: 110.000, steps: 563
Episode 16: reward: 80.000, steps: 528
Episode 17: reward: 135.000, steps: 468
Episode 18: reward: 120.000, steps: 538
Episode 19: reward: 275.000, steps: 788
Episode 20: reward: 355.000, steps: 1350


Average reward: 230.0

В този случай след само 100 000 стъпки агентът се е научил да играе по начин, с който бележи среден резултат от 230 точки на епизод, което е по-добро от 150-те точки при случаен избор на действие.

## Използвани публикации и литература
* https://nihit.github.io/resources/spaceinvaders.pdf
* https://arxiv.org/pdf/1312.5602.pdf